In [2]:
"""
Topic: Consistent Error Handling (using logger.error)
=====================================================
This script demonstrates how to use Python's logging module
for consistent and centralized error handling in ETL pipelines.
"""

# -------------------------------------------------------------------
# Slide 1 – Overview
# -------------------------------------------------------------------
"""
Consistent error handling ensures failures are logged, not hidden.
Use logging instead of print() to maintain visibility across jobs.

✅ Key Idea:
All exceptions should be handled with logger.error() or logger.critical().
"""

# -------------------------------------------------------------------
# Slide 2 – Imports and Logger Setup
# -------------------------------------------------------------------
import logging
import pandas as pd
import os
from datetime import datetime

# Configure logger
logging.basicConfig(
    filename="etl_error_handling.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

logger.info("🚀 Logger initialized successfully")

# -------------------------------------------------------------------
# Slide 3 – Simple Example with try/except
# -------------------------------------------------------------------
def read_input(file_path):
    """Read input CSV safely with error handling."""
    try:
        logger.info(f"Reading input file: {file_path}")
        df = pd.read_csv(file_path)
        logger.info(f"✅ Successfully read {len(df)} records.")
        return df
    except FileNotFoundError as e:
        logger.error(f"❌ File not found: {file_path} | Error: {e}")
    except Exception as e:
        logger.error(f"❌ Unexpected error while reading file: {e}")

# -------------------------------------------------------------------
# Slide 4 – Transformation Function
# -------------------------------------------------------------------
def transform(df):
    """Simulate transformation with exception handling."""
    try:
        logger.info("Starting transformation step...")
        if "price" not in df.columns:
            raise KeyError("Missing 'price' column")

        df["price_usd"] = df["price"] * 0.012
        df["processed_ts"] = datetime.now()
        logger.info("✅ Transformation completed.")
        return df
    except KeyError as e:
        logger.error(f"❌ Transformation failed: {e}")
    except Exception as e:
        logger.error(f"❌ Unknown error in transform(): {e}")

# -------------------------------------------------------------------
# Slide 5 – Persist Function
# -------------------------------------------------------------------
def persist(df, output_path):
    """Persist DataFrame to storage with logging."""
    try:
        logger.info(f"Saving DataFrame to {output_path}")
        # Simulate write
        if "price_usd" not in df.columns:
            raise ValueError("Data not transformed correctly")

        df.to_csv(output_path, index=False)
        logger.info(f"✅ Data saved successfully to {output_path}")
    except Exception as e:
        logger.error(f"❌ Failed to persist DataFrame: {e}")

# -------------------------------------------------------------------
# Slide 6 – Custom Error Example
# -------------------------------------------------------------------
def validate_records(df):
    """Validate that no price is zero or negative."""
    try:
        invalid = df[df["price"] <= 0]
        if not invalid.empty:
            raise ValueError(f"Invalid prices found: {len(invalid)} rows")
        logger.info("✅ Record validation passed.")
    except ValueError as e:
        logger.error(f"⚠️ Validation Error: {e}")
    except Exception as e:
        logger.error(f"❌ Unexpected error in validation: {e}")

# -------------------------------------------------------------------
# Slide 7 – Raising vs Logging Example
# -------------------------------------------------------------------
def safe_divide(a, b):
    """Example showing raise after log."""
    try:
        result = a / b
        logger.info(f"Result: {result}")
        return result
    except ZeroDivisionError as e:
        logger.error(f"❌ Division by zero: {e}")
        raise  # Re-raise after logging

# -------------------------------------------------------------------
# Slide 8 – Global Error Handling Wrapper
# -------------------------------------------------------------------
def main_pipeline():
    """Full ETL pipeline with consistent error logging."""
    try:
        df = read_input("orders.csv")  # Nonexistent to trigger log
        if df is not None:
            df = transform(df)
            validate_records(df)
            persist(df, "output/orders.csv")
        else:
            logger.warning("⚠️ Skipping transform step due to missing data.")
    except Exception as e:
        logger.critical(f"🔥 Pipeline crashed unexpectedly: {e}")

# -------------------------------------------------------------------
# Slide 9 – Real-World Pattern
# -------------------------------------------------------------------
"""
In real ETL:
1️⃣ Extract → Log API call status (200, 404, etc.)
2️⃣ Transform → Log bad rows, missing columns
3️⃣ Load → Log success/failure for each batch
All messages go to one structured log file.
"""

# -------------------------------------------------------------------
# Slide 10 – Best Practices & Summary
# -------------------------------------------------------------------
"""
✅ Best Practices:
- Initialize logger once per script
- Use logger.error() for handled exceptions
- Use logger.critical() for full job failures
- Include IDs, filenames, or batch references
- Never use print() for error messages
"""

# Run example
if __name__ == "__main__":
    logger.info("🏁 Starting error handling demo pipeline")
    main_pipeline()
    try:
        safe_divide(10, 0)
    except Exception:
        logger.info("Handled division error safely.")

    logger.info("✅ Script execution completed.")
